<a href="https://colab.research.google.com/github/NightPrinceY/CollegeForFun/blob/main/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the important liberaries for this project

In [ ]:
import json
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import pandas as pd
import requests


In [ ]:
#download the certain dataset for this project using url and requests library


In [ ]:

url = "https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json"
save_path = "sarcasm.json"

response = requests.get(url)
response.raise_for_status()

with open(save_path, "wb") as file:
    file.write(response.content)

print("Dataset downloaded successfully!")

In [ ]:


# Load the JSON file
with open('/kaggle/working/sarcasm.json', 'r') as f:
    datastore = json.load(f)

# Initialize the lists
sentences = []
labels = []

# Collect sentences and labels into the lists
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
sentences[2]

In [ ]:
#show the dataset as dataframe wtih show the first five examples

In [ ]:
dataset = pd.read_json('/kaggle/working/sarcasm.json')
dataset = pd.DataFrame(dataset)
dataset.head()


In [ ]:
headlines=pd.DataFrame(dataset['headline'])
headlines.head()

In [ ]:
labels = pd.DataFrame(dataset['is_sarcastic'] )
labels.head()

In [ ]:
# Number of examples to use for training
training_size = 20000

# Vocabulary size of the tokenizer
vocab_size = 10000

# Maximum length of the padded sequences
max_length = 32

# Output dimensions of the Embedding layer
embedding_dim = 16



In [ ]:
#split the data using slicing method
# yahya split the as trin= 20000 and val = remains = 6709

In [ ]:
# Split the sentences
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

# Split the labels
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [ ]:
#for the len of test size
len(testing_sentences)

In [ ]:
# Parameters for padding and OOV tokens
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
#creat tokenization with size words 10000 and ovv_tok as token for unkown
tokenizer = Tokenizer(num_words= vocab_size , oov_token = oov_tok)


In [ ]:
#runing tokenization on training data a
# sequnce and padd as final for the traing require
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
# the same for the test data and padd as final for val
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
# Convert the labels lists into numpy arrays
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

the coming code just for understand what happens on "GlobalAveragePooling1D (GAP1D) layer"
which it take the averagepool

In [ ]:

# Initialize a GlobalAveragePooling1D (GAP1D) layer
gap1d_layer = tf.keras.layers.GlobalAveragePooling1D()

# Define sample array
sample_array = np.array([[[10,2],[1,3],[1,1]]])

# Print shape and contents of sample array
print(f'shape of sample_array = {sample_array.shape}')
print(f'sample array: {sample_array}')

# Pass the sample array to the GAP1D layer
output = gap1d_layer(sample_array)

# Print shape and contents of the GAP1D output array
print(f'output shape of gap1d_layer: {output.shape}')
print(f'output array of gap1d_layer: {output.numpy()}')

In [ ]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30

# Train the model
history = model.fit(training_padded, training_labels,
                    epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels),
                    verbose=2)

In [ ]:
# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()


In [ ]:
# Plot the accuracy and loss
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

# Get the embedding layer from the model (i.e. first layer)
embedding_layer = model.layers[0]

# Get the weights of the embedding layer
embedding_weights = embedding_layer.get_weights()[0]

# Print the shape. Expected is (vocab_size, embedding_dim)
print(embedding_weights.shape)


i hope you enjoyed ..you can test the model with your words⬇️⬇️

In [ ]:
new = input("new_sentence: ")
new_review = [new]

# Tokenize and pad the new review
new_sequences = tokenizer.texts_to_sequences(new_review)
new_padded = pad_sequences(new_sequences, maxlen=max_length, truncating=trunc_type)

# Make a prediction using the trained model
prediction = model.predict(new_padded)
if prediction[0] > 0.5 :
    print('\n positive ')
else:
     print('\n negative ')
